In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_590170/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_49_9,0.997093,0.716747,0.709860,0.956097,0.012199,1.894111,1.037813,0.006664,0.227534,0.110449,1.001702,0.115152,138.812788,218.039717,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_49_8,0.997071,0.716814,0.709957,0.960335,0.012292,1.893668,1.037468,0.006021,0.220434,0.110868,1.001714,0.115588,138.797668,218.024597,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_49_7,0.997047,0.716880,0.710052,0.964396,0.012394,1.893222,1.037127,0.005404,0.213214,0.111329,1.001729,0.116069,138.781058,218.007987,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_49_6,0.997020,0.716948,0.710147,0.968270,0.012507,1.892772,1.036788,0.004816,0.205863,0.111835,1.001744,0.116596,138.762931,217.989860,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_49_5,0.996990,0.717016,0.710240,0.971949,0.012630,1.892318,1.036453,0.004258,0.198385,0.112385,1.001762,0.117170,138.743297,217.970225,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,model_1_4_4,0.890172,0.641847,0.660257,0.140174,0.460924,2.394972,1.215240,0.111894,0.574691,0.678914,1.064289,0.707816,131.549046,210.775974,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
456,model_1_4_3,0.888042,0.640249,0.659400,0.134929,0.469865,2.405660,1.218306,0.112577,0.579662,0.685467,1.065537,0.714649,131.510620,210.737549,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
457,model_1_4_2,0.885863,0.638645,0.658550,0.128741,0.479007,2.416386,1.221348,0.113382,0.584566,0.692103,1.066812,0.721567,131.472082,210.699011,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
458,model_1_4_1,0.883637,0.637037,0.657709,0.121711,0.488350,2.427138,1.224356,0.114297,0.589398,0.698820,1.068115,0.728570,131.433447,210.660376,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
